![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Flesson-plans&branch=master&subPath=notebooks/3Dprinting/3Dprinting-canoe.ipynb&depth=1"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# 3D Printing of Canoes

Credits:

- the canoe models orginate from a mathematical modeling project of Professor Cedric Chauve at Simon Fraser University
- the modelling code is available at this github repo: https://github.com/cchauve/CanoesProject_v2/tree/main
- the canoe designs are from the First Nations of the Pacific Northwest
- the 3D printing code is based on the Callysto notebook https://github.com/callysto/lesson-plans/tree/master/notebooks/3Dprinting

## Objective
In this notebook, we create an STL file of an Indigenous-style canoe suitable for printing on a 3D printer. 

The STL data is based on mathematical equations that representthe complex shape of canoes designed by First Nations people. The notebook includes an interactive tool to modify the dimension of the canoe before printing. 

## Different Types of Canoes

Ranging in length from three to twenty metres, canoes were essential for travel, transport, hunting, and trade. Different coastal communities developed distinctive styles to suit their particular needs. The following figures refer to canoes from the First Nations people of the Pacific Northwest Coast who are renowned for their elegantly engineered canoes.

The first figure shows the design of a Haida canoe, while the second figure shows the design specifications for the Salish canoe


<img src="images/haida00001a.gif" width="800">
<center>Canoe designs of the First Nations of the Pacific Northwest--Haida Canoe<center>
<p>
<font size="1"> 
image from: https://www.donsmaps.com/images30/haida00001a.gif
</font>
<p>


<img src="images/salish00001a.gif" width="800">
<center>Canoe designs of the First Nations of the Pacific Northwest--Salish Canoe<center>
<p>
<font size="1"> 
image from: https://www.donsmaps.com/images30/salish00001a.gif
</font>
<p>



## Building the 3D Canoe model

Run the following cell to build the canoe model and see it on the screen. Choose the canoe type, set the dimensions (or leave as the defaults), and click the run button to see your canoe. Adjust as necessary.

In [ ]:
import scripts.BezierSurf_Interface as bsi
bsi.Canoe() #NOOTKAN-STYLE Canoe

## Create the data arrays

The following code grabs the data from the canoe model and produces arrays X,Y,Z that are used to build the 3D model in the Standard Triangle Language (STL) file. 

In [ ]:
import numpy as np


Na,Nb = bsi.XX.shape

X= np.concatenate((bsi.XX,np.flip(bsi.XX[:,0:Nb-1],axis=1)),axis=1)
Y= np.concatenate((bsi.YY,np.flip(bsi.YY_mirror[:,0:Nb-1],axis=1)),axis=1)
Z= np.concatenate((bsi.ZZ,np.flip(bsi.ZZ[:,0:Nb-1],axis=1)),axis=1)


 ## Create the STL file
 
 We use the X,Y,Z arrays to produce a list of files triangles and save as an STL file called canoe.stl

 You may adjust the thickness of the canoe wall by setting the parameter h below. The default is h=0.1 which gives a rather thin-walled canoe.

In [ ]:
!pip install numpy-stl

In [ ]:
from stl import mesh

h = 0.1  # thickness for the solid surface
Z0 = Z - h # The bottom surface. Could also use Z0 = np.min(Z) for a flat base. 

Ny,Nx = X.shape
Nfaces = 4*(Nx-1)*(Ny-1) + 4*(Nx-1) + 4*(Ny-1)
surf = mesh.Mesh(np.zeros(Nfaces, dtype=mesh.Mesh.dtype))
k = 0

# top surface grid, two triangles per rectangular cell
for i in range(Ny-1):
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][2] = [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
        k += 1
# bottom surface grid, two triangles per rectangular cell. Note clockwise order!
for i in range(Ny-1):
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j+1],Y[i+1,j+1],Z0[i+1,j+1] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][1] = [ X[i+1,j+1],Y[i+1,j+1],Z0[i+1,j+1] ]
        k += 1

# now we create the side walls. Each wall connects the bottom surface Z0 to the top Z
# each wall has a line of rectangular cells which become two triangles each

# lower x side wall, two triangles for each of Nx-1 cells on edge
for i in [0]:
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][2] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        k += 1
# upper x side wall, note the clockwise order!
for i in [Ny-1]:
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][1] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        k += 1
# left y side wall, two triangles for each of Ny-1 cells on edge
for i in range(Ny-1):
    for j in [0]:
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        surf.vectors[k][2] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k += 1
# right y side wall, note the clockwise order!
for i in range(Ny-1):
    for j in [Nx-1]:
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        surf.vectors[k][1] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k += 1


# Write the mesh to file "canoe.stl"
surf.save('canoe.stl')

## Ready for printing!

You now have a file in your Jupyter hub called **canoe.stl** that you can print. Download this file to your local computer, then open it with your favourite 3D printing software. For instance, we have used PrusaSlicer, with is based on Slic3r. Use the Slicer software to adjust the size, add supports if necessary and adjust any other settings for the filament type and quality of print you require. Then create a g-code file and transfer this to your 3D printer. 

And print!  

Here is a sample result, using PLA filament in draft mode on a Prusa printer.

<div align="center">
<img src="images/canoe.jpeg" alt="The final printed canoe" width="600"/><br>
The final printed canoe.
</div>

## Going further

We can try several ideas to explore these canoes:

- Drag your STL file to view it at this website: https://www.viewstl.com/ 
- Try printing both types of canoes here, both Haida and Salish
- Try changing the dimensions of the canoe, to get some different variations
- Try changing the thickness of the wall of the canoe, by varying the parameter **h** mentioned above
- Can you design your own canoe?


[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)